![title](./pics/dd_logo.png) 

# Pipelines

In the [**configuration notebook**](./Analyzer_Configuration.ipynb), we have already touched on the structure of a pipeline. In this section, we will take a deeper look into its architecture.

As a first step, we will take a closer look at the **deep**doctection analyzer and lay the groundwork for developing custom pipelines tailored to specific use cases. After all, document parsing is just one possible application.

You might, for example, want to build a pipeline that classifies pages from a scanned binder and extracts entities from them. For such scenarios, **deep**doctection provides powerful components and tools that help you achieve your goals with a manageable amount of code.

If you go to the get started notebook and scroll to the cell where the image processed, you will see some logs:


```
[0523 22:14.35 @doctectionpipe.py:103]  INF  Processing sample_2.png
[0523 22:14.37 @context.py:133]         INF  ImageLayoutService total: 2.3095 sec.
[0523 22:14.37 @context.py:133]         INF  AnnotationNmsService total: 0.002 sec.
[0523 22:14.38 @context.py:133]         INF  SubImageLayoutService total: 0.3684 sec.
[0523 22:14.38 @context.py:133]         INF  PubtablesSegmentationService total: 0.0066 sec.
[0523 22:14.38 @context.py:133]         INF  ImageLayoutService total: 0.4052 sec.
[0523 22:14.39 @context.py:133]         INF  TextExtractionService total: 0.9374 sec.
[0523 22:14.39 @context.py:133]         INF  MatchingService total: 0.0059 sec.
[0523 22:14.39 @context.py:133]         INF  TextOrderService total: 0.0301 sec.
```

The logs reveal which pipeline component the image passed through and how much time each component took to process it. Some pipeline components have been deactivated by configuration.

A pipeline is built as a sequence of tasks. These tasks are called pipeline components or services.

![pipelines](./pics/dd_overview_pipeline.png)

Once a pipeline is defined, images or documents can be processed. These are either pure image files (like JPG, PNG, TIFF) or PDF files. PDF files are read and processed page by page. Each page is converted into a numpy array because arrays are the input for vision and OCR models. 


In [1]:
import deepdoctection as dd
from pathlib import Path

In [ ]:
analyzer = dd.get_dd_analyzer()

Let's take a closer look at the **deep**doctection analyzer. 

![pipeline](./pics/dd_pipeline_250525.png)

The architecture is modular, and a pipeline consists of individual components, each typically performing a single processing step. We have already explored the configuration options. When the analyzer is instantiated, a dictionary is printed to the logs, which begins approximately like this:

```bash
{'DEVICE': device(type='mps'),
 'LANGUAGE': None,
 'LAYOUT_LINK': {'CHILD_CATEGORIES': ['caption'], 'PARENTAL_CATEGORIES': ['figure', 'table']},
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': [['table', 'title'], ['table', 'text'],
                                       ['table', 'key_value_area'], ['table', 'list_item'],
                                       ['table', 'list'], ['table', 'figure'], ['title', 'text'],
                                       ['text', 'key_value_area'], ['text', 'list_item'],
                                       ['key_value_area', 'list_item']],
                      'PRIORITY': ['table', 'table', 'table', 'table', 'table', 'table', 'text',
                                   'text', None, 'key_value_area'],
                      'THRESHOLDS': [0.001, 0.01, 0.01, 0.001, 0.01, 0.01, 0.05, 0.01, 0.01, 0.01]},
 ...
}
```

Having a pipeline, you can list the components with `get_pipeline_info()`. It returns a dictionary with the a so called `service_id` and a name of the component. Note, that the name of the component depends not only on the service itself but also on the model that is being chosen.

In [3]:
analyzer.get_pipeline_info()

{'5497d92c': 'image_Transformers_Tatr_deformable-detr-DocLayNet_model.safetensors',
 '3b56c997': 'nms',
 'dc2cd45e': 'sub_image_Transformers_Tatr_tatr_tab_struct_v2_model.safetensors',
 '03844ddb': 'table_transformer_segment',
 '7d6a1bc2': 'image_doctr_db_resnet50db_resnet50_db_resnet50-ac60cadc.pt',
 '50d37ed6': 'text_extract_doctr_crnn_vgg16_bncrnn_vgg16_bn_crnn_vgg16_bn-0417f351.pt',
 'd6219eba': 'matching',
 'f10aa678': 'text_order'}

In [4]:
component = analyzer.get_pipeline_component(service_id="5497d92c")
component.name

'image_Transformers_Tatr_deformable-detr-DocLayNet_model.safetensors'

Each pipeline component has a `DatapointManager`, which manages the dataclass responsible for collecting all information related to a page. The results are then provided through the `Page` object, which generates the corresponding `JSON` output. If a service uses a model, the model will also receive a `model_id`. If you want to process a pile of documents with a pipeline, you can pass a `service_id` to the `analyze` method which allows you to version the run. The `service_id` will be added in your JSON output. Note, that the analyzer will not generate a `session_id` by itself.

The `get_meta_annotation()` method allows you to see which elements are being detected. We already mentioned this method in the [**More on parsing**](./Analyzer_More_On_Parsing.ipynb) notebook. It exists both at the level of individual pipeline components and at the level of the entire pipeline.
At the pipeline level, all `get_meta_annotation()` outputs from the individual components are aggregated.

In [5]:
component.dp_manager.model_id, component.dp_manager.service_id, component.dp_manager.session_id

('af516519', '5497d92c', None)

In [7]:
component.get_meta_annotation()

{'image_annotations': (<DefaultType.DEFAULT_TYPE>,
  <LayoutType.CAPTION>,
  <LayoutType.TEXT>,
  <LayoutType.TITLE>,
  <LayoutType.FOOTNOTE>,
  <LayoutType.FORMULA>,
  <LayoutType.LIST_ITEM>,
  <LayoutType.PAGE_FOOTER>,
  <LayoutType.PAGE_HEADER>,
  <LayoutType.FIGURE>,
  <LayoutType.SECTION_HEADER>,
  <LayoutType.TABLE>),
 'sub_categories': {},
 'relationships': {},
 'summaries': ()}

## Pipeline operations and parsing results

Let’s now take a look at the parsed results from a technical perspective.


In [ ]:
pdf_path = Path.cwd() / "sample/2312.13560.pdf"

df = analyzer.analyze(path=pdf_path, session_id="9999z99z", max_datapoints=3) # max_datapoints limits the number of samples to at most 3
df.reset_state()
all_results = list(df)

If we select any layout section or word, we can immediately see from the object in which component it was generated.


In [9]:
page_2 = all_results[1]

sample_layout_section = page_2.layouts[0] # get the first layout section
print(f"service_id: {sample_layout_section.service_id}, model_id: {sample_layout_section.model_id}, session_id: {sample_layout_section.session_id}")


service_id: 5497d92c, model_id: af516519, session_id: 9999z99z


In [10]:
sample_word = sample_layout_section.get_ordered_words()[2]
print(f"layout section text: {sample_layout_section.text} \n \n word text: {sample_word.characters}  service_id: {sample_word.service_id}, model_id: {sample_word.model_id}, session_id: {sample_word.session_id}")

layout section text: When processing audio at the frame level, an immense vol- ume of entries is generated, where a considerable portion of the frames are assigned to the "blank> " symbol due to the characteristic peak behavior of CTC. We propose a skip-blank strategy to prune the datastore and accelerate KNN retrieval. During datastore construction, this strategy omits frames whose CTC pseudo labels correspond to the "<blank>" symbol, thereby reducing the size of the data- store. This process is indicated by the blue dashed lines in 
 
 word text: audio  service_id: 7d6a1bc2, model_id: 8d10a01f, session_id: 9999z99z


A pipeline does not only generate layout sections but also determines other elements—such as reading orders or relationships between layout sections. The associated `service_id` must be extracted from the container that stores the reading order information.

In [11]:
reading_order = sample_layout_section.get_sub_category("reading_order")
print(type(reading_order))

<class 'dd_core.datapoint.annotation.CategoryAnnotation'>


In [12]:
print(f"position: {reading_order.category_id} \n \n service_id: {reading_order.service_id}  model_id: {reading_order.model_id}, session_id: {reading_order.session_id}")

position: 24 
 
 service_id: f10aa678  model_id: None, session_id: 9999z99z


We can get an overview of all `service_id`'s and their `annotation_id`'s they generated.

In [14]:
service_id_to_annotation_id = page_2.base_image.get_service_id_to_annotation_id()
service_id_to_annotation_id.keys(), service_id_to_annotation_id["01a15bff"][:10]

(dict_keys(['5497d92c', 'f10aa678', '7d6a1bc2', '50d37ed6', 'd6219eba', '01a15bff']),
 [])

Conversely, for a given `annotation_id`, we can use the `AnnotationMap` to locate the position where the object with that specific `annotation_id` can be found.
In the case below, the object with `annotation_id='db070af9-a38a-38cb-b157-9f4802842e5b'` is located within the layout section with `annotation_id='db070af9-a38a-38cb-b157-9f4802842e5b'` and represents the `sub_category = <Relationships.READING_ORDER>`.


In [20]:
annotation_id_to_annotation_maps = page_2.base_image.get_annotation_id_to_annotation_maps()
annotation_id_to_annotation_maps['db070af9-a38a-38cb-b157-9f4802842e5b']

[AnnotationMap(image_annotation_id='04a7b05b-165d-31eb-acf1-ab05269e34db', sub_category_key=<Relationships.READING_ORDER>, relationship_key=None, summary_key=None)]

We can retrieve the object in two steps:

In [21]:
image_annotation = page_2.get_annotation(annotation_ids='04a7b05b-165d-31eb-acf1-ab05269e34db')[0] # get the layout section
reading_order_ann = image_annotation.get_sub_category("reading_order") # get the reading order annotation
reading_order_ann.annotation_id # reading_order_ann is the object we were looking for

'db070af9-a38a-38cb-b157-9f4802842e5b'

## Un-doing a pipeline component operation

Not only the creation of objects but also the revision of a parsing structure can be important. In particular, the output of a component can be reverted using `undo`. In this example, we remove all word positions that were identified by the DocTr text detection model.

In [23]:
text_detection_component = analyzer.get_pipeline_component(service_id='5497d92c')

In [25]:
df = dd.DataFromList([dp.base_image for dp in all_results]) # Check the notebook Data_structure  in order to  
# understand, why we use dp.base_image.
df = text_detection_component.undo(df)
df.reset_state()

all_results_modified = list(df)

[1227 12:57.32 @context.py:156]  INF  ImageLayoutService total: 0.0591 sec.
[1227 12:57.33 @context.py:156]  INF  ImageLayoutService total: 0.0521 sec.
[1227 12:57.33 @context.py:156]  INF  ImageLayoutService total: 0.1483 sec.


Note, that not only the objects generated by the pipeline itself are affected, but also those from all pipeline components that build upon its results. In this case, objects related to text ordering and layout segments remain. The fact that text ordering objects persist might be surprising, but it can be explained by the fact that not only the words, but also the layout segments themselves are ordered. As a result, the reading order determined for these segments leads to the retention of objects from this service.


In [27]:
page_2_modified = all_results_modified[1]
service_id_to_annotation_id_modified = page_2_modified.get_service_id_to_annotation_id()
service_id_to_annotation_id_modified.keys(), service_id_to_annotation_id_modified.get('5497d92c')

(dict_keys(['dc2cd45e', '03844ddb', '7d6a1bc2', '50d37ed6', 'f10aa678', 'd6219eba']),
 None)

## Analyzer Factory


**How is an Analyzer constructed and where does the configuration come from?**

The configuration is provided by a default instance called `cfg`, which can be modified by defrosting it.







In [28]:
from deepdoctection.analyzer import cfg, ServiceFactory

cfg.USE_OCR

True

In [29]:
cfg.freeze(False)
cfg.USE_OCR=False  # After defrosting we can change values and add new attributes
cfg.freeze(True)


In [ ]:
cfg

```bash
{'CELL': {'FILTER': None,
          'PAD': {'BOTTOM': 60, 'LEFT': 60, 'RIGHT': 60, 'TOP': 60},
          'PADDING': False,
          'WEIGHTS': 'cell/d2_model_1849999_cell_inf_only.pt',
          'WEIGHTS_TS': 'cell/d2_model_1849999_cell_inf_only.ts'},
 'DEVICE': device(type='mps'),
 'ENFORCE_WEIGHTS': {'CELL': True, 'ITEM': True, 'LAYOUT': True},
 'ITEM': {'FILTER': ['table'],
          'PAD': {'BOTTOM': 60, 'LEFT': 60, 'RIGHT': 60, 'TOP': 60},
          'PADDING': False,
          'WEIGHTS': 'deepdoctection/tatr_tab_struct_v2/model.safetensors',
          'WEIGHTS_TS': 'item/d2_model_1639999_item_inf_only.ts'},
 'LANGUAGE': None,
 'LAYOUT': {'FILTER': None,
            'PAD': {'BOTTOM': 0, 'LEFT': 0, 'RIGHT': 0, 'TOP': 0},
            'PADDING': False,
            'WEIGHTS': 'Aryn/deformable-detr-DocLayNet/model.safetensors',
            'WEIGHTS_TS': 'layout/d2_model_0829999_layout_inf_only.ts'},
 'LAYOUT_LINK': {'CHILD_CATEGORIES': [<LayoutType.CAPTION>],
                 'PARENTAL_CATEGORIES': [<LayoutType.FIGURE>, <LayoutType.TABLE>]},
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': [[<LayoutType.TABLE>, <LayoutType.TITLE>],
                                       [<LayoutType.TABLE>, <LayoutType.TEXT>],
                                       [<LayoutType.TABLE>, <LayoutType.KEY_VALUE_AREA>],
                                       [<LayoutType.TABLE>, <LayoutType.LIST_ITEM>],
                                       [<LayoutType.TABLE>, <LayoutType.LIST>],
                                       [<LayoutType.TABLE>, <LayoutType.FIGURE>],
                                       [<LayoutType.TITLE>, <LayoutType.TEXT>],
                                       [<LayoutType.TEXT>, <LayoutType.KEY_VALUE_AREA>],
                                       [<LayoutType.TEXT>, <LayoutType.LIST_ITEM>],
                                       [<LayoutType.TEXT>, <LayoutType.CAPTION>],
                                       [<LayoutType.KEY_VALUE_AREA>, <LayoutType.LIST_ITEM>],
                                       [<LayoutType.FIGURE>, <LayoutType.CAPTION>]],
                      'PRIORITY': [<LayoutType.TABLE>, <LayoutType.TABLE>, <LayoutType.TABLE>,
                                   <LayoutType.TABLE>, <LayoutType.TABLE>, <LayoutType.TABLE>,
                                   <LayoutType.TEXT>, <LayoutType.TEXT>, None, <LayoutType.CAPTION>,
                                   <LayoutType.KEY_VALUE_AREA>, <LayoutType.FIGURE>],
                      'THRESHOLDS': [0.001, 0.01, 0.01, 0.001, 0.01, 0.01, 0.05, 0.01, 0.01, 0.01,
                                     0.01, 0.001]},
 'LIB': 'PT',
 'LM_LANGUAGE_DETECT_CLASS': {'WEIGHTS': None},
 'LM_SEQUENCE_CLASS': {'USE_OTHER_AS_DEFAULT_CATEGORY': False, 'WEIGHTS': None},
 'LM_TOKEN_CLASS': {'SEGMENT_POSITIONS': None,
                    'SLIDING_WINDOW_STRIDE': 0,
                    'USE_OTHER_AS_DEFAULT_CATEGORY': False,
                    'WEIGHTS': None},
 'OCR': {'CONFIG': {'TESSERACT': 'dd/conf_tesseract.yaml'},
         'USE_DOCTR': True,
         'USE_TESSERACT': False,
         'USE_TEXTRACT': False,
         'WEIGHTS': {'DOCTR_RECOGNITION': 'doctr/crnn_vgg16_bn/crnn_vgg16_bn-0417f351.pt',
                     'DOCTR_WORD': 'doctr/db_resnet50/db_resnet50-ac60cadc.pt'}},
 'PDF_MINER': {'X_TOLERANCE': 3, 'Y_TOLERANCE': 3},
 'ROTATOR': {'MODEL': 'tesseract'},
 'SEGMENTATION': {'ASSIGNMENT_RULE': 'ioa',
                  'CELL_NAMES': [<CellType.COLUMN_HEADER>, <CellType.BODY>, <LayoutType.CELL>],
                  'FULL_TABLE_TILING': True,
                  'ITEM_NAMES': [<LayoutType.ROW>, <LayoutType.COLUMN>],
                  'PUBTABLES_CELL_NAMES': [<LayoutType.CELL>],
                  'PUBTABLES_ITEM_HEADER_CELL_NAMES': [<CellType.COLUMN_HEADER>,
                                                       <CellType.ROW_HEADER>,
                                                       <CellType.PROJECTED_ROW_HEADER>],
                  'PUBTABLES_ITEM_HEADER_THRESHOLDS': [0.6, 0.0001],
                  'PUBTABLES_ITEM_NAMES': [<LayoutType.ROW>, <LayoutType.COLUMN>],
                  'PUBTABLES_SPANNING_CELL_NAMES': [<CellType.SPANNING>],
                  'PUBTABLES_SUB_ITEM_NAMES': [<CellType.ROW_NUMBER>, <CellType.COLUMN_NUMBER>],
                  'REMOVE_IOU_THRESHOLD_COLS': 0.2,
                  'REMOVE_IOU_THRESHOLD_ROWS': 0.2,
                  'STRETCH_RULE': 'equal',
                  'SUB_ITEM_NAMES': [<CellType.ROW_NUMBER>, <CellType.COLUMN_NUMBER>],
                  'TABLE_NAME': <LayoutType.TABLE>,
                  'THRESHOLD_COLS': 0.4,
                  'THRESHOLD_ROWS': 0.4},
 'TEXT_CONTAINER': <LayoutType.WORD>,
 'TEXT_ORDERING': {'BROKEN_LINE_TOLERANCE': 0.003,
                   'FLOATING_TEXT_BLOCK_CATEGORIES': (<LayoutType.TEXT>, <LayoutType.TITLE>,
                                                      <LayoutType.LIST>,
                                                      <LayoutType.KEY_VALUE_AREA>),
                   'HEIGHT_TOLERANCE': 2.0,
                   'INCLUDE_RESIDUAL_TEXT_CONTAINER': True,
                   'PARAGRAPH_BREAK': 0.035,
                   'STARTING_POINT_TOLERANCE': 0.005,
                   'TEXT_BLOCK_CATEGORIES': (<LayoutType.TEXT>, <LayoutType.TITLE>,
                                             <LayoutType.LIST_ITEM>, <LayoutType.LIST>,
                                             <LayoutType.CAPTION>, <LayoutType.PAGE_HEADER>,
                                             <LayoutType.PAGE_FOOTER>, <LayoutType.PAGE_NUMBER>,
                                             <LayoutType.MARK>, <LayoutType.KEY_VALUE_AREA>,
                                             <LayoutType.FIGURE>, <CellType.SPANNING>,
                                             <LayoutType.CELL>)},
 'USE_LAYOUT': True,
 'USE_LAYOUT_LINK': False,
 'USE_LAYOUT_NMS': True,
 'USE_LINE_MATCHER': False,
 'USE_LM_LANGUAGE_DETECTION': False,
 'USE_LM_SEQUENCE_CLASS': False,
 'USE_LM_TOKEN_CLASS': False,
 'USE_OCR': False,
 'USE_PDF_MINER': False,
 'USE_ROTATOR': False,
 'USE_TABLE_REFINEMENT': False,
 'USE_TABLE_SEGMENTATION': True,
 'WORD_MATCHING': {'MAX_PARENT_ONLY': True,
                   'PARENTAL_CATEGORIES': (<LayoutType.TEXT>, <LayoutType.TITLE>,
                                           <LayoutType.LIST_ITEM>, <LayoutType.LIST>,
                                           <LayoutType.CAPTION>, <LayoutType.PAGE_HEADER>,
                                           <LayoutType.PAGE_FOOTER>, <LayoutType.PAGE_NUMBER>,
                                           <LayoutType.MARK>, <LayoutType.KEY_VALUE_AREA>,
                                           <LayoutType.FIGURE>, <CellType.SPANNING>,
                                           <LayoutType.CELL>),
                   'RULE': 'ioa',
                   'THRESHOLD': 0.3}}
```

For constructing predictors (layout, table segmentation, OCR, etc.), pipeline components, and the default pipeline, a `ServiceFactory` offers a variety of methods.
We will not cover all the methods provided by the factory here, but rather give just one example. For a complete overview, please refer to the documentation.

In [33]:
#A very simple example of a pipeline includes a rotation detector that determines the rotation angle
# of a page (in multiples of 90 degrees) and rotates each page so that the text is in its correct,
# readable orientation.

rotation_detector = ServiceFactory.build_rotation_detector("tesseract")
transform_service = ServiceFactory.build_transform_service(transform_predictor=rotation_detector)
pipeline = dd.DoctectionPipe(pipeline_component_list=[transform_service])